In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
!unzip /content/drive/MyDrive/FruQ-DB/FruQ_MK2.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/FruQ-multi-new/content/FruQ-multi/BananaDB/Rotten/Image1001.png  
  inflating: content/FruQ-multi-new/content/FruQ-multi/BananaDB/Rotten/Image965.png  
  inflating: content/FruQ-multi-new/content/FruQ-multi/BananaDB/Rotten/Image979.png  
  inflating: content/FruQ-multi-new/content/FruQ-multi/BananaDB/Rotten/Image1275.png  
  inflating: content/FruQ-multi-new/content/FruQ-multi/BananaDB/Rotten/Image655.png  
  inflating: content/FruQ-multi-new/content/FruQ-multi/BananaDB/Rotten/Image1219.png  
  inflating: content/FruQ-multi-new/content/FruQ-multi/BananaDB/Rotten/Image569.png  
  inflating: content/FruQ-multi-new/content/FruQ-multi/BananaDB/Rotten/Image1267.png  
  inflating: content/FruQ-multi-new/content/FruQ-multi/BananaDB/Rotten/Image811.png  
  inflating: content/FruQ-multi-new/content/FruQ-multi/BananaDB/Rotten/Image1031.png  
  inflating: content/FruQ-multi-new/content/FruQ-multi/BananaDB/Rotten/Image775.png 

In [ ]:
import shutil

shutil.rmtree('/content/content/FruQ-multi-new/content/FruQ-multi/PepperQ')

In [ ]:
path = r'/content/content/FruQ-multi-new/content/FruQ-multi'

In [ ]:
# 1. Constants
IMG_SIZE = 224
BATCH_SIZE = 32
SEED = 123
FRUIT_CLASSES = ['AvocadoQ', 'BananaDB', 'CucumberQ', 'GrapefruitQ', 'KakiQ', 'PapayaQ', 'PeachQ', 'WatermeloQ', 'tomatoQ']

# Create dataset

In [ ]:
def create_data_generators():
    datagen = ImageDataGenerator(
        validation_split=0.2,
        # Geometric Augmentations
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=False,
        zoom_range=0.1,
        shear_range=0.2,
        # Color/Intensity Augmentations
        brightness_range=[0.8, 1.2],
        fill_mode='nearest'
    )

    val_datagen = ImageDataGenerator(validation_split=0.2)

    # Create a list to store all image paths and their fruit labels
    image_paths = []
    fruit_labels = []

    fruit_classes = FRUIT_CLASSES
    quality_classes = ['Fresh', 'Rotten', 'Mild']

    # Collect all images while maintaining their fruit class
    for fruit in fruit_classes:
        for quality in quality_classes:
            folder_path = os.path.join(path, fruit, quality)
            if os.path.exists(folder_path):
                for img in os.listdir(folder_path):
                    if img.endswith(('.jpg', '.jpeg', '.png')):
                        image_paths.append(os.path.join(folder_path, img))
                        fruit_labels.append(fruit)

    # Create a DataFrame to use with flow_from_dataframe
    import pandas as pd
    df = pd.DataFrame({
        'filename': image_paths,
        'class': fruit_labels
    })

    # Use flow_from_dataframe instead of flow_from_directory
    fruit_train = datagen.flow_from_dataframe(
        dataframe=df,
        x_col='filename',
        y_col='class',
        target_size=(IMG_SIZE,IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        seed=SEED  # Added seed
    )

    fruit_val = val_datagen.flow_from_dataframe(
        dataframe=df,
        x_col='filename',
        y_col='class',
        target_size=(IMG_SIZE,IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )

    # Similar process for quality classification
    quality_paths = []
    quality_labels = []

    for fruit in fruit_classes:
        for quality in quality_classes:
            folder_path = os.path.join(path, fruit, quality)
            if os.path.exists(folder_path):
                for img in os.listdir(folder_path):
                    if img.endswith(('.jpg', '.jpeg', '.png')):
                        quality_paths.append(os.path.join(folder_path, img))
                        quality_labels.append(quality)

    quality_df = pd.DataFrame({
        'filename': quality_paths,
        'class': quality_labels
    })

    quality_train = datagen.flow_from_dataframe(
        dataframe=quality_df,
        x_col='filename',
        y_col='class',
        target_size=(IMG_SIZE,IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        seed=SEED  # Added seed
    )

    quality_val = val_datagen.flow_from_dataframe(
        dataframe=quality_df,
        x_col='filename',
        y_col='class',
        target_size=(IMG_SIZE,IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )


    return (fruit_train, fruit_val), (quality_train, quality_val)

In [ ]:
(fruit_train, fruit_val), (quality_train, quality_val) = create_data_generators()

Found 6738 validated image filenames belonging to 9 classes.
Found 1684 validated image filenames belonging to 9 classes.
Found 6738 validated image filenames belonging to 3 classes.
Found 1684 validated image filenames belonging to 3 classes.


In [ ]:
# Add this after creating generators to check class distribution
print("Training set class distribution:")
print(fruit_train.classes)
print("\nClass indices:", fruit_train.class_indices)

# Check a batch of data and labels
images, labels = next(fruit_train)
print("\nBatch shape:", images.shape)
print("Labels shape:", labels.shape)
print("Sample labels:", labels[:5])

Training set class distribution:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

# 3. Model Creation

In [ ]:
def se_block(input_tensor, reduction_ratio=16):
    """
    Squeeze and Excitation block
    """
    channels = input_tensor.shape[-1]

    # Squeeze operation (global average pooling)
    x = layers.GlobalAveragePooling2D()(input_tensor)

    # Excitation operation (two FC layers)
    x = layers.Dense(channels // reduction_ratio, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(channels, activation='sigmoid')(x)

    # Reshape to broadcasting shape
    x = layers.Reshape((1, 1, channels))(x)

    # Scale the input
    return layers.multiply([input_tensor, x])

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

def create_model(num_classes, model_name=""):
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

     # Freeze base model
    base_model.trainable = False

    inputs = base_model.input

    # Proper preprocessing is crucial
    x = layers.Lambda(preprocess_input)(inputs)

    # Get base model features
    x = base_model(x, training=False)

    # Simple top layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)

    # Single dense layer with dropout
    x = layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.5)(x)


    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = Model(inputs=inputs, outputs=outputs, name=model_name)

    return model


# 4. Model Compilation

In [ ]:
def compile_model(model):
    model.compile(
        optimizer = tf.keras.optimizers.Adam(
        learning_rate=0.00001,  # Lower learning rate for more complex model
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=True  # Enable AMSGrad variant
    ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )



# 5. Callbacks


In [ ]:
def create_callbacks(model_name):
    return [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            min_delta=0.001
        ),
        tf.keras.callbacks.ModelCheckpoint(
            f'best_{model_name}.keras',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=1e-6
        )
    ]



# 6. Training History Plotting


In [ ]:
def plot_training_history(history, title):
    plt.figure(figsize=(12, 4))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{title} - Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{title} - Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])

    plt.tight_layout()
    plt.show()



# 7. Metric


In [ ]:
def metric(model, generator, class_names, title="Confusion Matrix"):
    # Get predictions
    generator.reset()
    y_true = generator.classes
    y_pred = model.predict(generator)
    y_pred = np.argmax(y_pred, axis=1)

    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Create figure and axis for confusion matrix
    plt.figure(figsize=(12, 8))

    # Create subplot layout - confusion matrix on left, metrics on right
    plt.subplot(1, 2, 1)

    # Plot confusion matrix
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=class_names,
        yticklabels=class_names
    )

    plt.title(f"{title}\nConfusion Matrix")
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

    # Calculate metrics
    report = classification_report(
        y_true,
        y_pred,
        target_names=class_names,
        output_dict=True
    )

    # Create metrics visualization
    plt.subplot(1, 2, 2)

    # Extract metrics for each class and macro average
    metrics_data = {
        'Precision': [],
        'Recall': [],
        'F1-Score': []
    }

    # Get metrics for each class
    for class_name in class_names:
        metrics_data['Precision'].append(report[class_name]['precision'])
        metrics_data['Recall'].append(report[class_name]['recall'])
        metrics_data['F1-Score'].append(report[class_name]['f1-score'])

    # Add macro average
    metrics_data['Precision'].append(report['macro avg']['precision'])
    metrics_data['Recall'].append(report['macro avg']['recall'])
    metrics_data['F1-Score'].append(report['macro avg']['f1-score'])

    # Create labels for all classes plus macro average
    all_labels = class_names + ['Macro Avg']

    # Create bar positions
    x = np.arange(len(all_labels))
    width = 0.25

    # Plot bars for each metric
    plt.bar(x - width, metrics_data['Precision'], width, label='Precision')
    plt.bar(x, metrics_data['Recall'], width, label='Recall')
    plt.bar(x + width, metrics_data['F1-Score'], width, label='F1-Score')

    plt.xlabel('Classes')
    plt.ylabel('Score')
    plt.title(f'{title}\nPrecision, Recall, and F1-Score')
    plt.xticks(x, all_labels, rotation=45)
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Print detailed classification report
    print("\nDetailed Classification Report:")
    print("\nPer-Class Metrics:")
    print("-" * 60)
    for class_name in class_names:
        print(f"\n{class_name}:")
        print(f"Precision: {report[class_name]['precision']:.3f}")
        print(f"Recall: {report[class_name]['recall']:.3f}")
        print(f"F1-Score: {report[class_name]['f1-score']:.3f}")

    print("\nMacro-Averaged Metrics:")
    print("-" * 60)
    print(f"Macro Precision: {report['macro avg']['precision']:.3f}")
    print(f"Macro Recall: {report['macro avg']['recall']:.3f}")
    print(f"Macro F1-Score: {report['macro avg']['f1-score']:.3f}")

    # Print support (number of samples per class)
    print("\nClass Support (Number of Samples):")
    print("-" * 60)
    for class_name in class_names:
        print(f"{class_name}: {report[class_name]['support']}")

In [ ]:
def evaluate_models(model, val, type):
    # Quality Classification Evaluation
    print("\nEvaluating  Classification Model...")
    if type.capitalize() == "Fruit":
        classes = os.listdir('FruitDB')
    elif type.capitalize() == "Quality":
        quality_classes = ['Fresh', 'Rotten', 'Mild']
    metric(
        quality_model,
        quality_val,
        classes,
        f"{type} Classification Confusion Matrix"
    )

# 8. Model Training Function

In [ ]:

def train_fruit_model(type):
    # Get data generators
    (fruit_train, fruit_val), _ = create_data_generators()

    # Create fruit classification model
    fruit_model = create_model(
        num_classes=len(FRUIT_CLASSES),
        model_name="fruit_classifier"
    )

    compile_model(fruit_model)

    # Add data sanity check
    train_sample = next(iter(fruit_train))[0]
    print("Sample input range:", tf.reduce_min(train_sample), "-", tf.reduce_max(train_sample))


    if type == "model":
        return fruit_model
    elif type == "train":
        print("Training Fruit Classification Model...")
        fruit_history = fruit_model.fit(
            fruit_train,
            validation_data=fruit_val,
            epochs=50,
            batch_size=32,
            verbose=1,
            callbacks=create_callbacks("fruit_classifier")
        )

        # Plot training history
        plot_training_history(fruit_history, "Fruit Classification")

        # Evaluate model
        print("\nEvaluating Fruit Classification Model...")
        metric(
            fruit_model,
            fruit_val,
            FRUIT_CLASSES,
            "Fruit Classification Confusion Matrix"
        )
        # Save model
        fruit_model.save('fruit_classifier_final.keras')
        print("Fruit model training completed and saved!")

        return fruit_history



In [ ]:
fruit_model= train_fruit_model("model")
fruit_model.summary()

Found 6738 validated image filenames belonging to 9 classes.
Found 1684 validated image filenames belonging to 9 classes.
Found 6738 validated image filenames belonging to 3 classes.
Found 1684 validated image filenames belonging to 3 classes.
Sample input range: tf.Tensor(0.0, shape=(), dtype=float32) - tf.Tensor(255.0, shape=(), dtype=float32)


Model: "fruit_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)          │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_12 (Lambda)                   │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_28          │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_64 (Dense)                     │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_65 (Dense)                     │ (None, 9)                   │           2,313 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,123,785 (92.02 MB)

 Trainable params: 531,465 (2.03 MB)

 Non-trainable params: 23,592,320 (90.00 MB)

In [ ]:
# Add this before training
print("\nChecking label encoding:")
print("Training set:")
print("Number of classes:", len(fruit_train.class_indices))
print("Class mapping:", fruit_train.class_indices)
print("Sample of class labels:", fruit_train.classes[:10])

# Check if the number of classes matches your Dense layer
assert fruit_model.layers[-1].units == len(fruit_train.class_indices), \
    "Mismatch between number of classes and output layer units"


Checking label encoding:
Training set:
Number of classes: 9
Class mapping: {'AvocadoQ': 0, 'BananaDB': 1, 'CucumberQ': 2, 'GrapefruitQ': 3, 'KakiQ': 4, 'PapayaQ': 5, 'PeachQ': 6, 'WatermeloQ': 7, 'tomatoQ': 8}
Sample of class labels: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
fruit_history = train_fruit_model("train")

Found 6738 validated image filenames belonging to 9 classes.
Found 1684 validated image filenames belonging to 9 classes.
Found 6738 validated image filenames belonging to 3 classes.
Found 1684 validated image filenames belonging to 3 classes.
Sample input range: tf.Tensor(0.0, shape=(), dtype=float32) - tf.Tensor(255.0, shape=(), dtype=float32)
Training Fruit Classification Model...
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


211/211 ━━━━━━━━━━━━━━━━━━━━ 294s 1s/step - accuracy: 0.2559 - loss: 6.7855 - val_accuracy: 0.0095 - val_loss: 6.7453 - learning_rate: 1.0000e-05
Epoch 2/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 270s 1s/step - accuracy: 0.8466 - loss: 5.1723 - val_accuracy: 0.0030 - val_loss: 6.9357 - learning_rate: 1.0000e-05
Epoch 3/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 322s 1s/step - accuracy: 0.9502 - loss: 4.7701 - val_accuracy: 0.0261 - val_loss: 6.7712 - learning_rate: 1.0000e-05
Epoch 4/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 272s 1s/step - accuracy: 0.9815 - loss: 4.5848 - val_accuracy: 0.1134 - val_loss: 6.5653 - learning_rate: 1.0000e-05
Epoch 5/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 272s 1s/step - accuracy: 0.9897 - loss: 4.4612 - val_accuracy: 0.2067 - val_loss: 6.3585 - learning_rate: 1.0000e-05
Epoch 6/50
198/211 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9957 - loss: 4.3607

KeyboardInterrupt: 

In [ ]:
plot_training_history(fruit_history, "Fruit Classification")

NameError: name 'fruit_history' is not defined

In [ ]:
evaluate_models(fruit_model, fruit_val, "fruit")

In [ ]:
# clear session for next train
tf.keras.backend.clear_session(
    free_memory=True
)

In [ ]:
def train_quality_model():
    # Get data generators
    _, (quality_train, quality_val) = create_data_generators()

    # Create quality classification model
    quality_model = create_model(
        num_classes=3,  # Fresh, Rotten, Mild
        model_name="quality_classifier"
    )
    compile_model(quality_model)

    print("Training Quality Classification Model...")
    quality_history = quality_model.fit(
        quality_train,
        validation_data=quality_val,
        epochs=50,
        callbacks=create_callbacks("quality_classifier")
    )

    # Plot training history
    plot_training_history(quality_history, "Quality Classification")

    # Evaluate model
    print("\nEvaluating Quality Classification Model...")
    quality_classes = ['Fresh', 'Rotten', 'Mild']
    create_confusion_matrix(
        quality_model,
        quality_val,
        quality_classes,
        "Quality Classification Confusion Matrix"
    )

    # Save model
    quality_model.save('quality_classifier_final.h5')
    print("Quality model training completed and saved!")

    return quality_model, quality_history

In [ ]:
quality_model, quality_history = train_quality_model()

In [ ]:
evaluate_models(quality_model, quality_val, "quality")

In [ ]:
# 10. Prediction Function
def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(
        image_path,
        target_size=(IMG_SIZE, IMG_SIZE)
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    return img_array / 255.0

def predict_image(image_path, fruit_model, quality_model):
    img = load_and_preprocess_image(image_path)

    fruit_pred = fruit_model.predict(img)
    quality_pred = quality_model.predict(img)

    fruit_classes = os.listdir(path)
    quality_classes = ['Fresh', 'Rotten', 'Mild']

    return {
        'fruit': fruit_classes[np.argmax(fruit_pred)],
        'quality': quality_classes[np.argmax(quality_pred)],
        'fruit_confidence': np.max(fruit_pred),
        'quality_confidence': np.max(quality_pred)
    }

